<a href="https://colab.research.google.com/github/Sirius-Aabhas/CS69002_9A_18CS60R55/blob/master/DL_Assign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [110]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as data_utils
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
device = torch.device('cuda:0')
device

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


device(type='cuda', index=0)

##Fetching data

In [0]:
from google.colab import files
import io
uploaded = files.upload()

In [0]:
type(uploaded), uploaded.keys(), type(uploaded['Train_20K.csv'])


In [0]:
df = pd.read_csv(io.StringIO(uploaded['Train_20K.csv'].decode('utf-8')), sep='\t')
df.head()

##Fetching data for now

In [111]:
df = pd.read_csv('Train_20K.csv', sep='\t')
df.tail()

,text,label
17994,I was pleasantly surprised by the film. Let's ...,1
17995,you must be seeing my comments over many films...,0
17996,This is one of those movies that they did too ...,1
17997,Anyone notice that Tommy only has 3 facial exp...,0
17998,I remember watching ATTACK when it first came ...,0


In [113]:
df_test = pd.read_csv('Test_5K.csv', sep='\t')
df_test.tail()

,text,label
4995,I don't know how to describe this movie. It's ...,0
4996,I found this movie hilarious. The spoofs on ot...,1
4997,My family and I have viewed this movie often o...,1
4998,"The Shining, you know what's weird about this ...",1
4999,Nobody could like this movie for its merit but...,0


In [114]:
print('Number of Negative movie reviews', len(df[df['label']==0]))
print('Number of Positive movie reviews', len(df[df['label']==1]))

Number of Negative movie reviews 8994
Number of Positive movie reviews 9005


In [161]:
text_reviews = df['text'].astype(str).tolist()
text_labels = df['label'].astype(int)

text_reviews = [x.lower() for x in text_reviews]

filtered_text_reviews = []
for sent in text_reviews:
    sent = sent.translate(str.maketrans('', '', string.punctuation))
    word_tokens = word_tokenize(sent)
    filtered_text_reviews.append(' '.join([w for w in word_tokens if (not w in stop_words and w != 'br')]))
            
text_reviews = filtered_text_reviews
print(text_reviews[0], text_labels[0])
print(len(text_reviews), len(text_labels))

john waters given us genuinely enjoyable film certainly isnt without shocking watersesque moments tamer older culty stuff pink flamingoes pecker harkens back johns early mainstream stage reminds viewer kind humor evident polyester overall really fun comedy great moments 1
17999 17999


In [162]:
data = [(text_reviews[i], text_labels[i])for i in range(len(text_labels))]
data[:5]

[('john waters given us genuinely enjoyable film certainly isnt without shocking watersesque moments tamer older culty stuff pink flamingoes pecker harkens back johns early mainstream stage reminds viewer kind humor evident polyester overall really fun comedy great moments',
  1),
 ('first two seasons comedy series strange werent funny drama element bill mother struggling usual problems life element bit depressing didnt mix well th comedy elements probably dropped soon became one funniest comedy series bbc ever made chemistry bill bens characters funny always many brilliant memorable sketches series christmas specials hilarious real treat christmas show came stop main actor gary olsen playing bill passed away sad brilliant actor films n funny man ripbr underrated show sadly disappeared television screens doesnt repeated often though appear uktv gold repeated bbc one two show brilliant comedy new audience',
  1),
 ('unfortunate mess shiner wanted like overthetop antifilm aspirant fact f

In [0]:
from google.colab import files
import io
uploaded = files.upload()

In [0]:
type(uploaded), uploaded.keys(), type(uploaded['Test_5K.csv'])

In [0]:
df_test = pd.read_csv(io.StringIO(uploaded['Test_5K.csv'].decode('utf-8')), sep='\t')
df_test.head()

In [117]:
print('Number of Negative movie reviews', len(df_test[df_test['label']==0]))
print('Number of Positive movie reviews', len(df_test[df_test['label']==1]))

Number of Negative movie reviews 2482
Number of Positive movie reviews 2518


In [163]:
text_reviews_test = df_test['text'].astype(str).tolist()
text_labels_test = df_test['label'].astype(int)

text_reviews_test = [x.lower() for x in text_reviews_test]

filtered_text_reviews = []
for sent in text_reviews_test:
    sent = sent.translate(str.maketrans('', '', string.punctuation))
    word_tokens = word_tokenize(sent)
    filtered_text_reviews.append(' '.join([w for w in word_tokens if (not w in stop_words and w != 'br')]))
            
text_reviews_test = filtered_text_reviews

print(text_reviews_test[0], text_labels_test[0])
print(len(text_reviews_test), len(text_labels_test))

always inaccurate picture homeless tv told lot lies panhandlers early 1990s made everyone look bad claimed made 100 day 2040 day much closer reality someone drove held sign offering work offered work actually went took work physically ableand would offered 100000 id damned sure invested apt prepaid least 2 years kept bank still left 1020000 nl 12 25 cash games casinos usually always win could win decent bankroll instead win 1000 month playing always minimum buying due wanting risk losing homeless cause didnt wan na risk spending money going broke sometimes 10002000 sock slept outside anyone wanting talk contact sevencard2003 yahoo messengeri admit different homeless people though due fact never drank smoke took drugs im longer homeless govt housing 177 month getting ssi spend time winning online poker mom sunflower diversified worked hard get ssi glad days hiding stage convention center casino night sleeping worrying getting caught security finally tv crew picked theyd lot sooner shame

In [164]:
data_test = [(text_reviews_test[i], text_labels_test[i])for i in range(len(text_labels_test))]
data_test[:5]

[('always inaccurate picture homeless tv told lot lies panhandlers early 1990s made everyone look bad claimed made 100 day 2040 day much closer reality someone drove held sign offering work offered work actually went took work physically ableand would offered 100000 id damned sure invested apt prepaid least 2 years kept bank still left 1020000 nl 12 25 cash games casinos usually always win could win decent bankroll instead win 1000 month playing always minimum buying due wanting risk losing homeless cause didnt wan na risk spending money going broke sometimes 10002000 sock slept outside anyone wanting talk contact sevencard2003 yahoo messengeri admit different homeless people though due fact never drank smoke took drugs im longer homeless govt housing 177 month getting ssi spend time winning online poker mom sunflower diversified worked hard get ssi glad days hiding stage convention center casino night sleeping worrying getting caught security finally tv crew picked theyd lot sooner sh

## Creating Bag Of Word (BOW) representation of sentences.

In [0]:
def generate_word_ids(dataset):
    word_to_ix = {}
    word_cntr = {}
    word_set = set()
    #print(dataset)
    for sent in dataset:
        for word in sent.split():
            if word not in word_cntr:
                word_cntr[word] = 1
            else:
                word_cntr[word] += 1
    
    for word in word_cntr:
        if word_cntr[word] >= 5:
            word_to_ix[word] = len(word_to_ix)
            
    word_to_ix['<UNKNOWN>'] = len(word_to_ix)
        
    return word_to_ix

In [166]:
word_to_ix = generate_word_ids(text_reviews + text_reviews_test)
VOCAB_SIZE = len(word_to_ix)
VOCAB_SIZE

30624

In [0]:
def make_bow_vector(sentence, word_to_ix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix))
    for word in sentence.split():
        if word not in word_to_ix:
            #raise ValueError('Word',word,' not present in the dictionary. Sorry!')
            vec[word_to_ix['<UNKNOWN>']]+=1
        else:
            vec[word_to_ix[word]]+=1
    return vec.view(1, -1)

def make_target(label):
    return torch.LongTensor([label])


In [167]:
t1 = time.time()
features = make_bow_vector(text_reviews[0], word_to_ix).to(device)
for i in range(1,len(text_reviews)):
    vec = make_bow_vector(text_reviews[i], word_to_ix).to(device)
    features = torch.cat((features,vec)).to(device)
    if i%1000 == 0:
        print(time.time() - t1)
        t1 = time.time()
        print(features.shape)
targets = torch.tensor(text_labels, dtype=torch.int).to(device)

print(time.time() - t1)
print(features.shape)
print(targets.shape)
train = data_utils.TensorDataset(features, targets)
train_loader = data_utils.DataLoader(train, batch_size=32, shuffle=True)

2.024198532104492
torch.Size([1001, 30624])
3.3038599491119385
torch.Size([2001, 30624])
5.116696119308472
torch.Size([3001, 30624])
7.366191387176514
torch.Size([4001, 30624])
9.376363039016724
torch.Size([5001, 30624])
11.328217029571533
torch.Size([6001, 30624])
13.20327091217041
torch.Size([7001, 30624])
15.336836099624634
torch.Size([8001, 30624])
17.288937091827393
torch.Size([9001, 30624])
19.24294900894165
torch.Size([10001, 30624])
21.191832542419434
torch.Size([11001, 30624])
23.146239519119263
torch.Size([12001, 30624])
25.09600257873535
torch.Size([13001, 30624])
170.03065967559814
torch.Size([14001, 30624])
187.80506491661072
torch.Size([15001, 30624])
211.9542031288147
torch.Size([16001, 30624])
225.83997082710266
torch.Size([17001, 30624])
240.9227602481842
torch.Size([17999, 30624])
torch.Size([17999])


In [201]:
features[0].shape

torch.Size([30624])

##Task1

In [0]:
class Model1A(nn.Module):
    def __init__(self, vocab_size):
        super(Model1A, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 50)
        self.lin2 = nn.Linear(50, 2)

    def forward(self, x):
        x = self.lin1(x)
        return F.softmax(self.lin2(x))

In [0]:
class Model1B(nn.Module):
    def __init__(self, vocab_size):
        super(Model1B, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 100)
        self.lin2 = nn.Linear(100, 2)

    def forward(self, x):
        x = self.lin1(x)
        return F.softmax(self.lin2(x))

In [0]:
class Model1C(nn.Module):
    def __init__(self, vocab_size):
        super(Model1C, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 200)
        self.lin2 = nn.Linear(200, 2)

    def forward(self, x):
        x = self.lin1(x)
        return F.softmax(self.lin2(x))

In [0]:
class Model1D(nn.Module):
    def __init__(self, vocab_size):
        super(Model1D, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 500)
        self.lin2 = nn.Linear(500, 2)

    def forward(self, x):
        x = self.lin1(x)
        return F.softmax(self.lin2(x))

In [0]:
model1A = Model1A(VOCAB_SIZE)
model1B = Model1B(VOCAB_SIZE)
model1C = Model1C(VOCAB_SIZE)
model1D = Model1D(VOCAB_SIZE)

In [0]:
for param in model1A.parameters():
    print(param,param.size())

##Task2

In [0]:
class Model2A(nn.Module):
    def __init__(self, vocab_size):
        super(Model2A, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 10)
        self.lin2 = nn.Linear(10, 10)
        self.lin3 = nn.Linear(10, 2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.lin2(x)
        return F.softmax(self.lin3(x))

In [0]:
class Model2B(nn.Module):
    def __init__(self, vocab_size):
        super(Model2B, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 20)
        self.lin2 = nn.Linear(20, 10)
        self.lin3 = nn.Linear(10, 2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.lin2(x)
        return F.softmax(self.lin3(x))

In [0]:
class Model2C(nn.Module):
    def __init__(self, vocab_size):
        super(Model2C, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 30)
        self.lin3 = nn.Linear(30, 2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.lin2(x)
        return F.softmax(self.lin3(x))

In [0]:
class Model2D(nn.Module):
    def __init__(self, vocab_size):
        super(Model2D, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 50)
        self.lin2 = nn.Linear(50, 50)
        self.lin3 = nn.Linear(50, 2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.lin2(x)
        return F.softmax(self.lin3(x))

In [0]:
class Model2E(nn.Module):
    def __init__(self, vocab_size):
        super(Model2E, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 100)
        self.lin2 = nn.Linear(100, 50)
        self.lin3 = nn.Linear(50, 2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.lin2(x)
        return F.softmax(self.lin3(x))

In [0]:
model2A = Model2A(VOCAB_SIZE)
model2B = Model2B(VOCAB_SIZE)
model2C = Model2C(VOCAB_SIZE)
model2D = Model2D(VOCAB_SIZE)
model2E = Model2E(VOCAB_SIZE)

##Task3

In [0]:
class Model3A(nn.Module):
    def __init__(self, vocab_size):
        super(Model3A, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 100)
        self.lin2 = nn.Linear(100, 50)
        self.lin3 = nn.Linear(50, 10)
        self.lin4 = nn.Linear(10, 2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.lin2(x)
        x = self.lin3(x)
        return F.softmax(self.lin4(x))

In [0]:
class Model3B(nn.Module):
    def __init__(self, vocab_size):
        super(Model3B, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 200)
        self.lin2 = nn.Linear(200, 100)
        self.lin3 = nn.Linear(100, 10)
        self.lin4 = nn.Linear(10, 2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.lin2(x)
        x = self.lin3(x)
        return F.softmax(self.lin4(x))

In [0]:
model3A = Model3A(VOCAB_SIZE)
model3B = Model3B(VOCAB_SIZE)

##Task4

In [0]:
class Model4A(nn.Module):
    def __init__(self, vocab_size):
        super(Model4A, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 20)
        self.drop_layer = nn.Dropout(p=0.4)
        self.lin3 = nn.Linear(20, 10)
        self.lin4 = nn.Linear(10,2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.lin2(x)
        x = self.drop_layer(x)
        x = self.lin3(x)
        return F.softmax(self.lin4(x))

In [0]:
class Model4B(nn.Module):
    def __init__(self, vocab_size):
        super(Model4B, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 100)
        self.drop_layer = nn.Dropout(p=0.4)
        self.lin2 = nn.Linear(100, 100)
        self.lin3 = nn.Linear(100,2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.drop_layer(x)
        x = self.lin2(x)
        return F.softmax(self.lin3(x))

In [0]:
class Model4C(nn.Module):
    def __init__(self, vocab_size):
        super(Model4C, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 100)
        self.drop_layer = nn.Dropout(p=0.4)
        self.lin2 = nn.Linear(100, 10)
        self.lin3 = nn.Linear(10,2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.drop_layer(x)
        x = self.lin2(x)
        return F.softmax(self.lin3(x))

In [0]:
model4A = Model4A(VOCAB_SIZE)
model4B = Model4B(VOCAB_SIZE)
model4C = Model4C(VOCAB_SIZE)

##Task5

In [0]:
class Model5A(nn.Module):
    def __init__(self, vocab_size):
        super(Model5A, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 20)
        self.drop_layer = nn.Dropout(p=0.4)
        self.lin3 = nn.Linear(20, 10)
        self.lin4 = nn.Linear(10,2)

    def forward(self, x): 
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = self.drop_layer(x)
        x = F.relu(self.lin3(x))
        return F.softmax(self.lin4(x))

In [0]:
class Model5B(nn.Module):
    def __init__(self, vocab_size):
        super(Model5B, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 20)
        self.drop_layer = nn.Dropout(p=0.4)
        self.lin3 = nn.Linear(20, 10)
        self.lin4 = nn.Linear(10,2)

    def forward(self, x): 
        x = F.tanh(self.lin1(x))
        x = F.tanh(self.lin2(x))
        x = self.drop_layer(x)
        x = F.tanh(self.lin3(x))
        return F.softmax(self.lin4(x))

In [0]:
class Model5C(nn.Module):
    def __init__(self, vocab_size):
        super(Model5C, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 20)
        self.drop_layer = nn.Dropout(p=0.4)
        self.lin3 = nn.Linear(20, 10)
        self.lin4 = nn.Linear(10,2)

    def forward(self, x): 
        x = F.sigmoid(self.lin1(x))
        x = F.sigmoid(self.lin2(x))
        x = self.drop_layer(x)
        x = F.sigmoid(self.lin3(x))
        return F.softmax(self.lin4(x))

In [0]:
model5A = Model5A(VOCAB_SIZE)
model5B = Model5B(VOCAB_SIZE)
model5C = Model5C(VOCAB_SIZE)

In [0]:
for param in model5A.parameters():
    print(param,param.size())

##Task7

In [0]:
class Model7A(nn.Module):
    def __init__(self, vocab_size):
        super(Model7A, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 20)
        self.lin3 = nn.Linear(20, 10)
        self.lin4 = nn.Linear(10,2)

    def forward(self, x): 
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = F.relu(self.lin3(x))
        return F.softmax(self.lin4(x))

In [0]:
class Model7B(nn.Module):
    def __init__(self, vocab_size):
        super(Model7B, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 20)
        self.lin3 = nn.Linear(20, 10)
        self.lin4 = nn.Linear(10,2)

    def forward(self, x): 
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = F.relu(self.lin3(x))
        return F.softmax(self.lin4(x))

In [0]:
class Model7C(nn.Module):
    def __init__(self, vocab_size):
        super(Model7C, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 20)
        self.lin3 = nn.Linear(20, 10)
        self.lin4 = nn.Linear(10,2)

    def forward(self, x): 
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = F.relu(self.lin3(x))
        return F.softmax(self.lin4(x))

In [0]:
model7A = Model7A(VOCAB_SIZE)
model7B = Model7B(VOCAB_SIZE)
model7C = Model7C(VOCAB_SIZE)

## Run code

In [0]:
def train(model, isGpu):
    if isGpu:
        model = model.to(device)
    # define a loss function and an optimizer
    #loss_function = nn.NLLLoss()
    loss_function = nn.CrossEntropyLoss()
    opt = torch.optim.SGD(model.parameters(), lr = 0.1)
    # the training loop
    for epoch in range(30):
        tic = time.time()
        for batch_idx, (instance, label) in enumerate(train_loader):
            # get the training data
            model.zero_grad()
            label = label.long()
            probs = model(instance) # forward pass
            loss = loss_function(probs, label)
            loss.backward()
            opt.step()
        toc = time.time()
        print('EPOCH: {}, CURRENT LOSS: {}, TIME TAKEN: {}'.format(epoch, loss.data, (toc-tic)))


In [0]:
def validate(model):
    model = model.cpu()
    preds = []
    for instance, label in data_test:
        bow_vec = Variable(make_bow_vector(instance, word_to_ix))
        logprobs = model(bow_vec)
        #print(logprobs)
        pred = np.argmax(logprobs.data.cpu().numpy())
        preds.append(pred)

    print('accuracy: {}'.format(accuracy_score(text_labels_test, preds)))
    print('precision: {}'.format(precision_score(text_labels_test, preds)))
    print('recall: {}'.format(recall_score(text_labels_test, preds)))
    print('f1-score: {}'.format(f1_score(text_labels_test, preds)))


In [0]:
def run_and_save(model): 
    train(model, True)
    validate(model)
    torch.save(model, model._get_name()+'.mdl')

In [198]:
run_and_save(model1A)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


EPOCH: 0, CURRENT LOSS: 0.49253320693969727, TIME TAKEN: 0.8663370609283447
EPOCH: 1, CURRENT LOSS: 0.4049370586872101, TIME TAKEN: 0.8625960350036621
EPOCH: 2, CURRENT LOSS: 0.44335511326789856, TIME TAKEN: 0.8416721820831299
EPOCH: 3, CURRENT LOSS: 0.4035332500934601, TIME TAKEN: 0.7402517795562744
EPOCH: 4, CURRENT LOSS: 0.4415443539619446, TIME TAKEN: 0.7349205017089844
EPOCH: 5, CURRENT LOSS: 0.41690030694007874, TIME TAKEN: 0.7394413948059082
EPOCH: 6, CURRENT LOSS: 0.3134815990924835, TIME TAKEN: 0.7379934787750244
EPOCH: 7, CURRENT LOSS: 0.40137526392936707, TIME TAKEN: 0.7404682636260986
EPOCH: 8, CURRENT LOSS: 0.32842808961868286, TIME TAKEN: 0.8037106990814209
EPOCH: 9, CURRENT LOSS: 0.40754494071006775, TIME TAKEN: 0.7976877689361572
EPOCH: 10, CURRENT LOSS: 0.3270855247974396, TIME TAKEN: 0.796971321105957
EPOCH: 11, CURRENT LOSS: 0.41997823119163513, TIME TAKEN: 0.7877819538116455
EPOCH: 12, CURRENT LOSS: 0.3205462098121643, TIME TAKEN: 0.8043563365936279
EPOCH: 13, CURRE

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Model1A. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [199]:
run_and_save(model1B)
run_and_save(model1C)
run_and_save(model1D)
run_and_save(model2A)
run_and_save(model2B)
run_and_save(model2C)
run_and_save(model2D)
run_and_save(model2E)
run_and_save(model3A)
run_and_save(model3B)
run_and_save(model4A)
run_and_save(model4B)
run_and_save(model4C)
run_and_save(model5A)
run_and_save(model5B)
run_and_save(model5C)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


EPOCH: 0, CURRENT LOSS: 0.4651903212070465, TIME TAKEN: 0.9849138259887695
EPOCH: 1, CURRENT LOSS: 0.43574467301368713, TIME TAKEN: 0.8667426109313965
EPOCH: 2, CURRENT LOSS: 0.3871213495731354, TIME TAKEN: 0.8575284481048584
EPOCH: 3, CURRENT LOSS: 0.36921584606170654, TIME TAKEN: 0.8541278839111328
EPOCH: 4, CURRENT LOSS: 0.3813888430595398, TIME TAKEN: 0.8550138473510742
EPOCH: 5, CURRENT LOSS: 0.31594160199165344, TIME TAKEN: 0.8549597263336182
EPOCH: 6, CURRENT LOSS: 0.3977689743041992, TIME TAKEN: 0.8568789958953857
EPOCH: 7, CURRENT LOSS: 0.31725195050239563, TIME TAKEN: 0.851722240447998
EPOCH: 8, CURRENT LOSS: 0.3200514018535614, TIME TAKEN: 0.8576686382293701
EPOCH: 9, CURRENT LOSS: 0.3155335783958435, TIME TAKEN: 0.856403112411499
EPOCH: 10, CURRENT LOSS: 0.42370137572288513, TIME TAKEN: 0.8577849864959717
EPOCH: 11, CURRENT LOSS: 0.38885733485221863, TIME TAKEN: 0.8560278415679932
EPOCH: 12, CURRENT LOSS: 0.36514949798583984, TIME TAKEN: 0.8566176891326904
EPOCH: 13, CURREN

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Model1B. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


EPOCH: 0, CURRENT LOSS: 0.43329036235809326, TIME TAKEN: 1.5475678443908691
EPOCH: 1, CURRENT LOSS: 0.36012929677963257, TIME TAKEN: 1.4276082515716553
EPOCH: 2, CURRENT LOSS: 0.38632506132125854, TIME TAKEN: 1.43056321144104
EPOCH: 3, CURRENT LOSS: 0.32468318939208984, TIME TAKEN: 1.428098201751709
EPOCH: 4, CURRENT LOSS: 0.44756805896759033, TIME TAKEN: 1.4268500804901123
EPOCH: 5, CURRENT LOSS: 0.44814857840538025, TIME TAKEN: 1.4288887977600098
EPOCH: 6, CURRENT LOSS: 0.3242698013782501, TIME TAKEN: 1.4301447868347168
EPOCH: 7, CURRENT LOSS: 0.4683738052845001, TIME TAKEN: 1.429511547088623
EPOCH: 8, CURRENT LOSS: 0.37214699387550354, TIME TAKEN: 1.4311931133270264
EPOCH: 9, CURRENT LOSS: 0.3845963180065155, TIME TAKEN: 1.4268698692321777
EPOCH: 10, CURRENT LOSS: 0.37175002694129944, TIME TAKEN: 1.4248929023742676
EPOCH: 11, CURRENT LOSS: 0.313803493976593, TIME TAKEN: 1.422123670578003
EPOCH: 12, CURRENT LOSS: 0.39632999897003174, TIME TAKEN: 1.4298532009124756
EPOCH: 13, CURRENT 

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Model1C. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


...

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Model7A. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


EPOCH: 0, CURRENT LOSS: 0.5904111862182617, TIME TAKEN: 1.0195605754852295
EPOCH: 1, CURRENT LOSS: 0.45038992166519165, TIME TAKEN: 1.006758213043213
EPOCH: 2, CURRENT LOSS: 0.3857126533985138, TIME TAKEN: 1.009338617324829
EPOCH: 3, CURRENT LOSS: 0.3808403015136719, TIME TAKEN: 1.0106191635131836
EPOCH: 4, CURRENT LOSS: 0.47566378116607666, TIME TAKEN: 1.011232852935791
EPOCH: 5, CURRENT LOSS: 0.4285326302051544, TIME TAKEN: 1.0132973194122314
EPOCH: 6, CURRENT LOSS: 0.507980465888977, TIME TAKEN: 1.0046637058258057
EPOCH: 7, CURRENT LOSS: 0.489962637424469, TIME TAKEN: 1.0081157684326172
EPOCH: 8, CURRENT LOSS: 0.3139623999595642, TIME TAKEN: 1.0113391876220703
EPOCH: 9, CURRENT LOSS: 0.3710188865661621, TIME TAKEN: 1.0115845203399658
EPOCH: 10, CURRENT LOSS: 0.31501108407974243, TIME TAKEN: 1.0130786895751953
EPOCH: 11, CURRENT LOSS: 0.35876771807670593, TIME TAKEN: 1.0078303813934326
EPOCH: 12, CURRENT LOSS: 0.31350526213645935, TIME TAKEN: 1.004371166229248
EPOCH: 13, CURRENT LOSS

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Model7B. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


EPOCH: 0, CURRENT LOSS: 0.4369955360889435, TIME TAKEN: 1.0206513404846191
EPOCH: 1, CURRENT LOSS: 0.4234408438205719, TIME TAKEN: 1.0068368911743164
EPOCH: 2, CURRENT LOSS: 0.5136333703994751, TIME TAKEN: 1.010990858078003
EPOCH: 3, CURRENT LOSS: 0.5131607055664062, TIME TAKEN: 1.0046420097351074
EPOCH: 4, CURRENT LOSS: 0.51754230260849, TIME TAKEN: 1.0104196071624756
EPOCH: 5, CURRENT LOSS: 0.37348267436027527, TIME TAKEN: 1.0032331943511963
EPOCH: 6, CURRENT LOSS: 0.35635676980018616, TIME TAKEN: 1.0061759948730469
EPOCH: 7, CURRENT LOSS: 0.3177572190761566, TIME TAKEN: 1.0051546096801758
EPOCH: 8, CURRENT LOSS: 0.4328053295612335, TIME TAKEN: 1.009512186050415
EPOCH: 9, CURRENT LOSS: 0.38060396909713745, TIME TAKEN: 1.0092813968658447
EPOCH: 10, CURRENT LOSS: 0.5168617963790894, TIME TAKEN: 1.087310791015625
EPOCH: 11, CURRENT LOSS: 0.3211657702922821, TIME TAKEN: 1.082632303237915
EPOCH: 12, CURRENT LOSS: 0.38013219833374023, TIME TAKEN: 1.0754954814910889
EPOCH: 13, CURRENT LOSS:

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Model7C. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
mdl_1a = torch.load('Model7B.mdl')

In [0]:
validate(mdl_1a)

In [204]:
list = np.arange(15).reshape(3,-1)
list

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

In [0]:
a = torch.tensor(list)

In [214]:
list1 = np.arange(5)
list2 = np.arange(5,10)
list3 = np.arange(10,15)
list1 = torch.tensor(list1)
list2 = torch.tensor(list2)
list3 = torch.tensor(list3)
list = [list1, list2, list3]
list

[tensor([0, 1, 2, 3, 4]),
 tensor([5, 6, 7, 8, 9]),
 tensor([10, 11, 12, 13, 14])]

In [217]:
list1.numpy()

array([0, 1, 2, 3, 4])

In [218]:
list = [list1.numpy(), list2.numpy(), list3.numpy()]
list

[array([0, 1, 2, 3, 4]), array([5, 6, 7, 8, 9]), array([10, 11, 12, 13, 14])]